In [12]:
def evaluate_for_key(key):
    resultData = pd.read_csv('results/' + key + '.csv')     
    bugs = resultData.bug.unique()
    print("Project " + key)
    print(len(bugs))
    count1 = 0;
    count5 = 0;
    count10 = 0;
    mapValue = 0;
    mrr = 0;
    for bug in bugs:
        subData = resultData[resultData['bug'] == bug]
        subDataSorted = subData.sort_values('Result2',ascending=False)
        
        if subDataSorted.iloc[0,-2] == 1:
            count1 += 1;
        if 1 in subDataSorted.values[0:5,-2]:
            count5 += 1;
        if 1 in subDataSorted.values[0:10,-2]:
            count10 += 1;
        
        summe = 0;
        rr = 0;
        retrieved_d = 0;
        i = 0
        c = 0
        for resultV in subDataSorted.values[:,-2]:
            if resultV == 1:
                c += 1
                retrieved_d += 1;
                summe += retrieved_d /(i+1)
            if resultV == 1 and rr == 0:    
                rr += (1/ (i+1))
            i += 1
        if c != 0:
            mapValue += (summe/c)
        else: 
            print("c zero!")
            mapValue +=0
            #rr = 1
        mrr += rr
    return count1/len(bugs),count5/len(bugs),count10/len(bugs),mapValue/len(bugs),mrr/len(bugs)

In [2]:
import os
import pandas as pd

data_mp = pd.read_csv('statistic.csv')


data = pd.DataFrame()
for file in os.listdir("amalgam"):
    project, version = file.replace('smartshark_','').replace('_output.txt','').split("_")
    element = data_mp.loc[(data_mp['Project'] == "smartshark_" + project) & (data_mp['Version'] == version), 'Bugs'].values[0]
    data = data.append({"project" : project, "version" : version, "bugs": element}, ignore_index=True)

In [3]:
agg_funcs = {'bugs' : 'sum', 
         'version' : 'count'
}

data = data.groupby("project").agg(agg_funcs)
data


,bugs,version
project,,
ant-ivy,433.0,6
archiva,323.0,7
calcite,610.0,16
cayenne,330.0,2
commons-bcel,47.0,6
commons-beanutils,39.0,10
commons-codec,70.0,11
commons-collections,101.0,9
commons-compress,143.0,17


In [14]:
from pandas.errors import EmptyDataError

def evaluateForAlgo(base):
    data_map = dict()
    data_mrr = dict()
    data_hit1 = dict()
    data_hit5 = dict()
    data_hit10 = dict()
    
    for file in os.listdir(base):
        project, version = file.replace('smartshark_','').replace('_output.txt','').split("_")
        if project not in data_map:
            data_map[project] = 0
            data_mrr[project] = 0
            data_hit1[project] = 0
            data_hit5[project] = 0
            data_hit10[project] = 0
        try:
            data_mp = pd.read_csv(base + "/" + file, header=None,delim_whitespace=True)  
            a_group = data_mp.groupby(0)
            for name, group in a_group:
                rr = 0;
                    
                summe = 0;
                retrieved_d = 0;   
                c = 0
                best_value = -1;
                for index, element in group.iterrows():
                    if element[2] < best_value or best_value == -1:
                        best_value = element[2]
                    c += 1
                    retrieved_d += 1;
                    summe += retrieved_d /(element[2]+1)
                if best_value != -1:
                    data_mrr[project] += (1/ (best_value+1))
                data_map[project] += (summe/c)
                
                # These outputs are zero based!
                if best_value != -1 and best_value <= 0:
                    data_hit1[project] += 1
                if best_value != -1 and best_value <= 4:
                    data_hit5[project] += 1
                if best_value != -1 and best_value <= 9:
                    data_hit10[project] += 1
        except EmptyDataError:
             data_mp = pd.DataFrame()
    

    #data_mrr[project] = (1/ (data_mp[2]+1))    
    #print(data_mp)
    #c += 1
    #retrieved_d += 1;
    #summe += retrieved_d /(i+1)
    data['MAP_' + base] = pd.Series(data_map) /data["bugs"]
    data['MRR_' + base] =  pd.Series(data_mrr) / data["bugs"]
    data['HIT1_' + base] =  pd.Series(data_hit1) / data["bugs"]
    data['HIT5_' + base] =  pd.Series(data_hit5) / data["bugs"]
    data['HIT10_' + base] =  pd.Series(data_hit10) / data["bugs"]

In [9]:
evaluateForAlgo('buglocator')
evaluateForAlgo('bluir')
evaluateForAlgo('brtracer')
evaluateForAlgo('amalgam')
evaluateForAlgo('blia')
evaluateForAlgo('locus')
evaluateForAlgo('blizzard')
data.to_csv('results_other.csv')
data

,bugs,version,MAP_blizzard,MRR_blizzard,HIT1_blizzard,HIT5_blizzard,HIT10_blizzard,MAP_buglocator,MRR_buglocator,HIT1_buglocator,...,MAP_blia,MRR_blia,HIT1_blia,HIT5_blia,HIT10_blia,MAP_locus,MRR_locus,HIT1_locus,HIT5_locus,HIT10_locus
project,,,,,,,,,,,,,,,,,,,,,
ant-ivy,433.0,6,0.286776,0.284760,0.374134,0.681293,0.775982,0.391392,0.515149,0.535797,...,0.478723,0.582512,0.605081,0.801386,0.856813,0.429868,0.534466,0.528868,0.711316,0.787529
archiva,323.0,7,0.188035,0.187087,0.250774,0.448916,0.532508,0.222778,0.311970,0.309598,...,0.247384,0.305051,0.291022,0.486068,0.551084,0.230255,0.319562,0.325077,0.479876,0.563467
calcite,610.0,16,0.331816,0.323384,0.468852,0.763934,0.842623,0.338103,0.546393,0.547541,...,0.390813,0.563897,0.565574,0.785246,0.865574,0.395189,0.591865,0.601639,0.765574,0.819672
cayenne,330.0,2,0.233743,0.230929,0.293939,0.569697,0.669697,0.295288,0.419797,0.427273,...,0.327780,0.436438,0.421212,0.615152,0.703030,0.256835,0.350663,0.345455,0.493939,0.551515
commons-bcel,47.0,6,0.204366,0.199891,0.276596,0.468085,0.553191,0.301997,0.381283,0.382979,...,0.328937,0.386525,0.361702,0.446809,0.468085,0.331213,0.371972,0.404255,0.489362,0.531915
commons-beanutils,39.0,10,0.392814,0.385714,0.487179,0.948718,0.974359,0.481297,0.691350,0.717949,...,0.618250,0.757258,0.794872,0.846154,0.948718,0.419538,0.508974,0.512821,0.615385,0.641026
commons-codec,70.0,11,0.427763,0.417143,0.671429,0.957143,0.971429,0.645511,0.812738,0.871429,...,0.825368,0.880952,0.957143,0.985714,0.985714,0.564127,0.632200,0.657143,0.671429,0.728571
commons-collections,101.0,9,0.162714,0.156841,0.247525,0.356436,0.366337,0.231728,0.323763,0.336634,...,0.275521,0.356266,0.366337,0.376238,0.386139,0.239247,0.273079,0.297030,0.306931,0.316832
commons-compress,143.0,17,0.369665,0.375406,0.608392,0.825175,0.909091,0.585744,0.749731,0.776224,...,0.687885,0.787729,0.839161,0.944056,0.993007,0.624933,0.780187,0.811189,0.909091,0.972028


In [17]:
import os
import pandas as pd

data_mp = pd.read_csv('../other/statistic.csv')
data_mp

,Project,Version,Bugs,Result
0,Apache_CAMEL,CAMEL_1_6_0,299,NaN
1,Apache_CAMEL,CAMEL_1_6_1,39,NaN
2,Apache_CAMEL,CAMEL_1_6_2,6,NaN
3,Apache_CAMEL,CAMEL_1_6_3,2,NaN
4,Apache_CAMEL,CAMEL_1_6_4,67,NaN
...,...,...,...,...
706,Wildfly_WFLY,WFLY_8_2_1,2,NaN
707,Wildfly_WFLY,WFLY_9_0_0,74,NaN
708,Wildfly_WFLY,WFLY_9_0_1,5,NaN
709,Wildfly_WFLY,WFLY_9_0_2,5,NaN


In [34]:
data = pd.DataFrame()
for file in os.listdir("../other/amalgam"):
    project = "_".join(file.replace('_output.txt','').split("_")[:2])
    version = "_".join(file.replace('_output.txt','').split("_")[2:])
    #print(project,version)
    element = data_mp.loc[(data_mp['Project'] == project) & (data_mp['Version'] == version), 'Bugs'].values[0]
    data = data.append({"project" : project, "version" : version, "bugs": element}, ignore_index=True)

In [35]:
agg_funcs = {'bugs' : 'sum', 
         'version' : 'count'
}

data = data.groupby("project").agg(agg_funcs)

data

,bugs,version
project,,
Apache_CAMEL,1452.0,62
Apache_HBASE,858.0,70
Apache_HIVE,1219.0,21
Commons_CODEC,41.0,9
Commons_COLLECTIONS,93.0,7
Commons_COMPRESS,111.0,15
Commons_CONFIGURATION,135.0,11
Commons_CRYPTO,8.0,1
Commons_CSV,14.0,3


In [39]:
from pandas.errors import EmptyDataError

def evaluateForAlgo(base, factor=0):
    data_map = dict()
    data_mrr = dict()
    data_hit1 = dict()
    data_hit5 = dict()
    data_hit10 = dict()
    
    for file in os.listdir('../other/' + base):
        print(file)
        project = "_".join(file.replace('_output.txt','').split("_")[:2])
        version = "_".join(file.replace('_output.txt','').split("_")[2:])
        if project not in data_map:
            data_map[project] = 0
            data_mrr[project] = 0
            data_hit1[project] = 0
            data_hit5[project] = 0
            data_hit10[project] = 0
        try:
            data_mp = pd.read_csv('../other/' + base + "/" + file, header=None,error_bad_lines=False,sep='\t')  
            a_group = data_mp.groupby(0)
            for name, group in a_group:
                rr = 0;
                    
                summe = 0;
                retrieved_d = 0;   
                c = 0
                best_value = -1;
                for index, element in group.iterrows():
                    if element[2] < best_value or best_value == -1:
                        best_value = element[2]
                    c += 1
                    retrieved_d += 1;
                    summe += retrieved_d /(element[2]+1)
                if best_value != -1:
                    data_mrr[project] += (1/ (best_value+1))
                data_map[project] += (summe/c)
                # These outputs are zero based!
                #print(best_value)
                if best_value != -1 and best_value == 0 + factor:
                    data_hit1[project] += 1
                if best_value != -1 and best_value <= 4 + factor:
                    data_hit5[project] += 1
                if best_value != -1 and best_value <= 9 + factor:
                    data_hit10[project] += 1
        except EmptyDataError:
             data_mp = pd.DataFrame()
    

    #data_mrr[project] = (1/ (data_mp[2]+1))    
    #print(data_mp)
    #c += 1
    #retrieved_d += 1;
    #summe += retrieved_d /(i+1)
    data['MAP_' + base] = pd.Series(data_map) /data["bugs"]
    data['MRR_' + base] =  pd.Series(data_mrr) / data["bugs"]
    data['HIT1_' + base] =  pd.Series(data_hit1) / data["bugs"]
    data['HIT5_' + base] =  pd.Series(data_hit5) / data["bugs"]
    data['HIT10_' + base] =  pd.Series(data_hit10) / data["bugs"]


evaluateForAlgo('buglocator')
evaluateForAlgo('bluir')
evaluateForAlgo('brtracer')
evaluateForAlgo('amalgam')
evaluateForAlgo('blia')
evaluateForAlgo('locus')
evaluateForAlgo('blizzard', 1)
data.to_csv('results_other_bench4bl.csv')
data

Apache_CAMEL_CAMEL_2_13_0_output.txt
Spring_DATAMONGO_DATAMONGO_1_3_3_output.txt
Spring_DATAREST_DATAREST_2_1_6_output.txt
Spring_BATCH_BATCH_3_0_3_output.txt
Spring_SWF_SWF_2_0_5_output.txt
Apache_HBASE_HBASE_0_99_2_output.txt
Apache_CAMEL_CAMEL_2_9_4_output.txt
Spring_DATAGRAPH_DATAGRAPH_4_1_1_output.txt
Spring_DATAJPA_DATAJPA_1_7_3_output.txt
Commons_MATH_MATH_3_3_output.txt
Commons_COMPRESS_COMPRESS_1_12_output.txt
Spring_DATACMNS_DATACMNS_1_3_2_output.txt
Spring_DATAREDIS_DATAREDIS_1_4_4_output.txt
Apache_CAMEL_CAMEL_2_16_2_output.txt
Spring_DATAJPA_DATAJPA_1_4_3_output.txt
Spring_ANDROID_ANDROID_1_0_1_output.txt
Spring_SWS_SWS_1_0_2_output.txt
Commons_COMPRESS_COMPRESS_1_1_output.txt
JBoss_JBMETA_JBMETA_8_0_0_output.txt
Spring_SWS_SWS_2_1_2_output.txt
Apache_HBASE_HBASE_1_2_0_output.txt
Apache_HBASE_HBASE_0_99_0_output.txt
Spring_DATAJPA_DATAJPA_1_0_3_output.txt
Commons_CODEC_CODEC_1_3_output.txt
Spring_DATACMNS_DATACMNS_1_5_3_output.txt
Spring_SEC_SEC_3_2_7_output.txt
Apache_HIV

,bugs,version,MAP_buglocator,MRR_buglocator,HIT1_buglocator,HIT5_buglocator,HIT10_buglocator,MAP_bluir,MRR_bluir,HIT1_bluir,...,MAP_locus,MRR_locus,HIT1_locus,HIT5_locus,HIT10_locus,MAP_blizzard,MRR_blizzard,HIT1_blizzard,HIT5_blizzard,HIT10_blizzard
project,,,,,,,,,,,,,,,,,,,,,
Apache_CAMEL,1452.0,62,0.376696,0.466102,0.358127,0.588843,0.668733,0.215749,0.215884,0.125344,...,0.468673,0.559306,0.439394,0.707300,0.783058,0.209513,0.207293,0.278926,0.502755,0.570248
Apache_HBASE,858.0,70,0.368868,0.452408,0.325175,0.606061,0.691142,0.252355,0.260922,0.165501,...,0.463303,0.553281,0.441725,0.698135,0.769231,0.282126,0.278388,0.396270,0.665501,0.727273
Apache_HIVE,1219.0,21,0.276611,0.371965,0.250205,0.507793,0.605414,0.213947,0.216826,0.139459,...,0.359737,0.454056,0.334701,0.596390,0.701395,0.219608,0.216124,0.274815,0.521739,0.632486
Commons_CODEC,41.0,9,0.768098,0.847339,0.756098,0.975610,0.975610,0.703630,0.707046,0.560976,...,0.737634,0.781010,0.658537,0.951220,0.951220,0.427676,0.425813,0.756098,0.902439,0.926829
Commons_COLLECTIONS,93.0,7,0.588393,0.631820,0.505376,0.763441,0.827957,0.378350,0.363965,0.247312,...,0.702980,0.735188,0.677419,0.806452,0.838710,0.346136,0.339956,0.494624,0.806452,0.827957
Commons_COMPRESS,111.0,15,0.656692,0.733429,0.603604,0.909910,0.963964,0.390213,0.398244,0.216216,...,0.705223,0.787693,0.693694,0.918919,0.990991,0.369854,0.369249,0.603604,0.810811,0.918919
Commons_CONFIGURATION,135.0,11,0.651569,0.706430,0.592593,0.859259,0.896296,0.389311,0.393490,0.288889,...,0.572495,0.607351,0.444444,0.800000,0.881481,0.330659,0.332878,0.503704,0.762963,0.822222
Commons_CRYPTO,8.0,1,0.243720,0.244213,0.125000,0.500000,0.500000,0.184375,0.184375,0.125000,...,0.381819,0.458333,0.375000,0.625000,0.625000,0.156250,0.138889,0.250000,0.250000,0.375000
Commons_CSV,14.0,3,0.652888,0.674317,0.571429,0.785714,0.857143,0.579167,0.588690,0.500000,...,0.636395,0.660204,0.500000,0.857143,1.000000,0.440476,0.428571,0.642857,1.000000,1.000000
